In [51]:
from astropy import time
import numpy as np
from IPython import display
import pandas as pd
import panel as pn
pn.extension("tabulator")
import hvplot.pandas
import holoviews as hv

from lwautils import lwa_arx
from lwautils import TimeoutException
from lwa_antpos import reading, mapping
from mnc import mcs, common, control
import dsautils.cnf as cnf
from dsautils import dsa_store
from lwa_f import snap2_fengine, snap2_feng_etcd_client
from lwa352_pipeline_control import Lwa352PipelineControl, Lwa352CorrelatorControl


ls = dsa_store.DsaStore()

INFO:{"mjd": 59927.73149137398, "proj": "dsa", "subsystem": "System", "app": "_", "version": "_", "module": "dsaStore", "function": "_check_host", "level": "info", "time": "2022-12-14T17:33:20.854704Z", "msg": "TODO: implement"}
INFO:{"mjd": 59927.731491559476, "proj": "dsa", "subsystem": "System", "app": "_", "version": "_", "module": "dsaStore", "function": "_check_port", "level": "info", "time": "2022-12-14T17:33:20.870731Z", "msg": "TODO: implement"}
INFO:{"mjd": 59927.731491659826, "proj": "dsa", "subsystem": "System", "app": "_", "version": "_", "module": "dsaStore", "function": "c-tor", "level": "info", "time": "2022-12-14T17:33:20.879404Z", "msg": "DsaStore created"}


## Set up

In [52]:
con = control.Controller("/home/pipeline/proj/lwa-shell/mnc_python/config/lwa_config_calim.yaml")

Loaded configuration for 8 x-engine host(s) running 4 pipeline(s) each
Supported modes are: slow,beam1,beam2,beam3,beam4
etcd server being used is: etcdv3service


In [53]:
dd_statuses = []

## cnf

In [54]:
df_cnf_read = reading.read_antpos_etcd()
df_cnf = df_cnf_read[['snap2_hostname', 'snap2_chassis', 'snap2_location', 'fmc', 'pola_digitizer_channel',
                      'polb_digitizer_channel', 'arx_address', 'pola_arx_channel', 'polb_arx_channel',
                      'pola_fee', 'polb_fee']]
df_cnf_pane = pn.widgets.Tabulator(df_cnf, layout='fit_data_table', pagination='remote', page_size=20)

## Build table functions

In [55]:
# arx table build function
def build_a():
    ma = lwa_arx.ARX()
    arxcfg = []

    adrs = con.conf['arx']['adrs']
    for adr in adrs:
        try:
            dds = ma.get_all_chan_cfg(adr)
        except:
            continue
        for ch, dd in enumerate(dds):
            dd['adr-ch'] = f'{adr}-{ch}'
            arxcfg.append(dd)
        dd_astatus = {'name': f'arx{adr}', 'mp_age': 0, 'ready': dd['sig_on']}
        dd_statuses.append(dd_astatus)

    df_arx = pd.DataFrame.from_dict(arxcfg)
    if any(df_arx):
        df_arx.set_index('adr-ch', inplace=True)

    df_a_pane = pn.widgets.Tabulator(df_arx, layout='fit_data_table', pagination='remote',
                                     width=900, page_size=20)
    return df_a_pane


# f-engine table build function
def build_f():
    dd_fs = []
    t_now = time.Time.now().unix
    stats = con.status_fengine()

    snap2names = con.conf['fengines']['snap2s_inuse']
    for snap2name in snap2names:
        snap2num = int(snap2name.lstrip('snap'))

        t_now = time.Time.now().unix
        if stats[snap2name] is None:
            continue
        else:
            dd_fstatus = {'name': snap2name, 'age (s)': t_now-time.Time(stats[snap2name]['fpga']['timestamp']).unix,
                          'ready': stats[snap2name]['fpga']['programmed']}

        dd_statuses.append(dd_fstatus)

        plot_fstats = None

        dd_f = {'name': snap2name}
        st = stats[snap2name]['fpga']
        dd_f['age (s)'] = int(t_now-time.Time(st['timestamp']).unix)
#    dd_f['is_connected'] = lwa_feng.is_connected()  # can SNAP be reached with ping on network?
#    dd_f['is_polling'] = lwa_fe.is_polling()   # is monitor service checking on SNAP?
        dd_f['programmed'] = st['programmed']
        dd_f['firmware'] = st['flash_firmware']
        

        st = stats[snap2name]['sync']
        dd_f['uptime_fpga_clks'] = st['uptime_fpga_clks']

        st = stats[snap2name]['autocorr']
        dd_f['autocorr_acc_len'] = st['acc_len']

        st = stats[snap2name]['corr']
        dd_f['corr_acc_len'] = st['acc_len']

        st = stats[snap2name]['eqtvg']
        dd_f['tvg_enabled'] = st['tvg_enabled']

        st = stats[snap2name]['eq']
        dd_f['clip_count'] = st['clip_count']

        dd_fs.append(dd_f)

    df_f = pd.DataFrame.from_dict(dd_fs)
    if any(df_f):
        df_f.set_index('name', inplace=True)
        
    df_f_pane = pn.widgets.Tabulator(df_f, layout='fit_data_table', pagination='remote',
                                     width=900, page_size=20)

    return df_f_pane


# TODO: f-engine stats plot build function
def build_fstats(snap2name):
    st = stats[snap2name]['input']
    pows = np.array([v for (k,v) in st.items() if 'pow' in k])
    means = np.array([v for (k,v) in st.items() if 'mean' in k])
    rms = np.array([v for (k,v) in st.items() if 'rms' in k])
#    snap_input = [f'{snap2name}-{k.strip("rms")}' for (k,v) in st.items() if 'rms' in k]
    df_fstats = pd.DataFrame(data={'pows': pows, 'mean': means, 'snap': len(pows)*[f'{snap2name}']})
    plot_fstats = df_fstats.hvplot.scatter('pows', 'mean', by='snap', width=500, title='F-engine stats')

    return plot_fstats


# x-engine table build function
def build_x():
    npipeline = con.conf['xengines']['nxpipeline']
    dd_xs = []
    t_now = time.Time.now().unix

    xhosts = con.conf['xengines']['xhosts']
    for host in xhosts:
        for pipeline in range(npipeline):
            lwa_x = Lwa352PipelineControl(pipeline_id=pipeline, etcdhost=common.ETCD_HOST, host=host)
            st = lwa_x.corr.get_bifrost_status()
            stats = st['stats']
            t_now = time.Time.now().unix
            name = f'{host}-{pipeline}'
            dd_x = {'name': name}
            dd_x['mp_age'] = t_now-st['time']
            dd_x['state'] = stats['state']
            if lwa_x.pipeline_is_up():
#                dd_xstatus = {'name': name}
#                dd_xstatus['age (s)'] = int(t_now-st['time'])
                dd_x['gbps'] = st['gbps']

                if len(stats) > 1:
#                    dd_xstatus['ready'] = stats['state'] == 'running'
                    if stats['state'] == 'running':
                        dd_x['curr_sample'] = stats['curr_sample']
#                dd_statuses.append(dd_xstatus)
            else:
                print(f'Pipeline for {host}:{pipeline} is not up')

            dd_xs.append(dd_x)

    df_x = pd.DataFrame.from_dict(dd_xs)
    if any(df_x):
        df_x.set_index('name', inplace=True)

    df_x_pane = pn.widgets.Tabulator(df_x, layout='fit_data_table', pagination='remote',
                                     width=900, page_size=20)

    return df_x_pane


# Data recorder table build function
def build_dr():
    dd_ds = []
    t_now = time.Time.now().unix
    data_baselines = None
    data_spectra = None
    
    recorders = con.conf['dr']['recorders'].copy()
    if 'drvs' in recorders:
        recorders.remove('drvs')
        for num in con.drvnums[::2]:  # one per pair
            recorders.append('drvs'+str(num))

    if 'drvf' in recorders:
        recorders.remove('drvf')
        for num in con.drvnums[::2]:  # one per pair
            recorders.append('drvf'+str(num))

    for drid in recorders:
        lwa_dr = mcs.Client(drid)
        summary = lwa_dr.read_monitor_point('summary')
        if summary is None:
            continue
        t_mp = summary.timestamp
        dd_dstatus = {'name': drid, 'age (s)': t_now-t_mp,
                      'ready': summary.value == 'normal'}
        dd_statuses.append(dd_dstatus)

        rx_rate = None
        rx_missing = None
        pipeline_lag = None
        if 'bifrost/rx_rate' in lwa_dr.list_monitor_points():
            rx_rate = lwa_dr.read_monitor_point('bifrost/rx_rate').value
            rx_missing = lwa_dr.read_monitor_point('bifrost/rx_missing').value
            pipeline_lag = lwa_dr.read_monitor_point('bifrost/pipeline_lag').value
            load_average1 = lwa_dr.read_monitor_point('system/load_average/one_minute').value
            tbfree = lwa_dr.read_monitor_point('storage/active_disk_free').value/1e12
            activedir = lwa_dr.read_monitor_point('storage/active_directory').value

        infostring = lwa_dr.read_monitor_point('info').value + ', ' + lwa_dr.read_monitor_point('op-type').value

        dd_d = {'name': f'{drid}', 'age (s)': int(t_now-t_mp), 'rx_rate': int(rx_rate), #'rx_miss': rx_missing,
                'pipeline_lag': pipeline_lag, 'load_1m': load_average1,
                'Disk free (TB)': tbfree, 'Active dir': activedir,
                'info': infostring}
        dd_ds.append(dd_d)
    else:
        lwa_dr = None
        
    df_d = pd.DataFrame.from_dict(dd_ds)
    if any(df_d):
        df_d.set_index('name', inplace=True)

    df_d_pane = pn.widgets.Tabulator(df_d, layout='fit_data_table', pagination='remote',
                                     width=900, page_size=20)

    return df_d_pane


def build_plot(antnum):
    if isinstance(antnum, int) and antnum > 0:
        plot_pane = pn.pane.JPG(f'/home/ubuntu/data/plots/{antnum:03}.jpg', width=900)
    else:
        plot_pane = None

    return plot_pane


def build_drvbl(drname):
    if drname not in ['drvs'+str(num) for num in con.drvnums]:
        pane_baselines = None
    else:
        cl = mcs.Client(drname)
        data_baselines = cl.read_monitor_point('diagnostics/baselines').value
        pane_baselines = pn.pane.PNG(display.Image(mcs.ImageMonitorPoint._decode_image_data(data_baselines)),
                                     width=900)
    return pane_baselines

In [56]:
#df_arx_pane
#df_f_pane
#df_x_pane
#df_d_pane
# pane_baselines

In [57]:
#df_status = pd.DataFrame.from_dict(dd_statuses)
#df_status.set_index('name', inplace=True)

## Build dashboard

In [58]:
ant_filter = pn.widgets.IntInput(name='Select ant for f-eng summary plot',
                                 placeholder='1,2,3,4,5,6,7,8,9,10,...,352')
dr_filter = pn.widgets.TextInput(name='Select dr for baseline plot',
                                 placeholder=','.join(['drvs'+str(num) for num in con.drvnums]))

# TODO: make menu work
#dr_filter = pn.widgets.MenuButton(items=['drvs'+str(num) for num in con.drvnums])

pa = pn.param.ParamFunction(build_a, lazy=True)
pf = pn.param.ParamFunction(build_f, lazy=True)
px = pn.param.ParamFunction(build_x, lazy=True)
pdr = pn.param.ParamFunction(build_dr, lazy=True)

In [59]:
dashboard_title = pn.panel('## OVRO-LWA System Health Dashboard')
tabs = pn.Tabs(#('Status', df_status_pane),
               ('Cabling', df_cnf_pane),
               ('ARX', pa),
               ('F-engine', pf),
               ('X-engine', px),
               ('Data recorder', pdr), dynamic=True, active=4)

header_f = pn.pane.Markdown('##Plots', width=500)
plot_spec_pane = pn.Column(pn.Column(ant_filter), pn.panel(pn.bind(build_plot, ant_filter)), width=500)
plot_dr_pane = pn.Column(pn.Column(dr_filter), pn.panel(pn.bind(build_drvbl, dr_filter)), width=500)
mini_dashboard = pn.Column(dashboard_title, tabs, header_f, plot_spec_pane, plot_dr_pane, width=500)

In [61]:
mini_dashboard.servable(title='OVRO-LWA dashboard')

Column(width=500)
    [0] Markdown(str)
    [1] Tabs(active=4, dynamic=True)
        [0] Tabulator(pagination='remote', value=         snap2_hostname  ...)
        [1] ParamFunction(function, lazy=True)
        [2] ParamFunction(function, lazy=True)
        [3] ParamFunction(function, lazy=True)
        [4] ParamFunction(function, lazy=True)
    [2] Markdown(str, width=500)
    [3] Column(width=500)
        [0] Column
            [0] IntInput(name='Select ant for f..., placeholder='1,2,3,4,5,6,7...)
        [1] ParamFunction(function)
    [4] Column(width=500)
        [0] Column
            [0] TextInput(name='Select dr for b..., placeholder='drvs7601,drvs7601,d...)
        [1] ParamFunction(function)